In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import community as community_louvain
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
# Função para clusterização baseada em grafos
def graph_based_clustering(data, min_size, max_size):
    # Construir um grafo a partir dos dados
    G = nx.Graph()
    for i in data.index:
        G.add_node(i, latitude=data.at[i, 'LATITUDE'], longitude=data.at[i, 'LONGITUDE'])
    
    # Adicionar arestas com base na similaridade entre os pontos (distância euclidiana)
    coords = data[['LATITUDE', 'LONGITUDE']].values
    distances = euclidean_distances(coords, coords)
    for i in range(len(data)):
        for j in range(i+1, len(data)):
            similarity = 1 / (1 + distances[i, j])  # Inverso da distância para similaridade
            G.add_edge(i, j, weight=similarity)
    
    # Executar a detecção de comunidades otimizando a modularidade
    partition = community_louvain.best_partition(G)
    
    # Construir clusters a partir dos resultados da detecção de comunidades
    clusters = {}
    for node, cluster_id in partition.items():
        if cluster_id not in clusters:
            clusters[cluster_id] = []
        clusters[cluster_id].append(node)
    
    # Filtrar clusters que não atendem às restrições de tamanho
    filtered_clusters = {cluster_id: nodes for cluster_id, nodes in clusters.items()
                         if min_size <= len(nodes) <= max_size}
    
    return filtered_clusters

In [ ]:
# Leitura dos dados
data = pd.read_csv("../data_source/amostra_total.csv", sep=';')
data = data[["INDICE", "LATITUDE", "LONGITUDE", "LOGRADOURO", "NUMERO"]]

In [ ]:
# Definir tamanhos mínimo e máximo de cluster
min_size = 300
max_size = 400

In [ ]:
# Executar a clusterização
clusters = graph_based_clustering(data, min_size, max_size)

In [ ]:
# Exibir resultados
for cluster_id, nodes in clusters.items():
    print(f"Cluster {cluster_id}: {len(nodes)} pontos")
    print(data.loc[nodes])